In [ ]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv
from tqdm.auto import tqdm


In [ ]:
def load_triples(path):
    triples = []
    with open(path) as f:
        for line in f:
            h, r, t = line.strip().split()[:3]
            triples.append((h, r, t))
    return triples

train_triples = load_triples("../graph_data/train.tsv")
dev_triples =  load_triples("../graph_data/dev.tsv")
test_triples =  load_triples("../graph_data/test.tsv")



entities = sorted({e for (h, _, t) in train_triples + dev_triples + test_triples for e in (h, t)})
relations = sorted({r for (_, r, _) in train_triples + dev_triples + test_triples})
ent2id = {e: i for i, e in enumerate(entities)}
rel2id = {r: i for i, r in enumerate(relations)}
num_nodes = len(entities)
num_rels = len(relations)


edge_index, edge_type = [[], []], []
for h, r, t in train_triples + dev_triples:  # include dev for embedding
    edge_index[0].append(ent2id[h]); edge_index[1].append(ent2id[t])
    edge_type.append(rel2id[r])
edge_index = torch.tensor(edge_index, dtype=torch.long)
edge_type  = torch.tensor(edge_type,  dtype=torch.long)
data = Data(num_nodes=num_nodes, edge_index=edge_index, edge_type=edge_type)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data = data.to(device)


class RGCNClassifier(nn.Module):
    def __init__(self, num_nodes, num_rels, hidden_dim, num_layers=2, dropout=0.3):
        super().__init__()

        self.convs = nn.ModuleList()
        self.convs.append(RGCNConv(num_nodes, hidden_dim, num_rels, num_bases=30))
        for _ in range(num_layers-1):
            self.convs.append(RGCNConv(hidden_dim, hidden_dim, num_rels, num_bases=30))
        self.dropout = dropout

        self.classifier = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_rels)
        )

    def forward(self, x, edge_index, edge_type, h_idx, t_idx):

        for conv in self.convs:
            x = conv(x, edge_index, edge_type)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        h_emb = x[h_idx]
        t_emb = x[t_idx]
        # concatenate and classify
        combined = torch.cat([h_emb, t_emb], dim=-1)
        return self.classifier(combined)



def encode_cls(triples):
    return torch.tensor([[ent2id[h], ent2id[t], rel2id[r]] for h, r, t in triples], dtype=torch.long)

train_data = encode_cls(train_triples)
dev_data   = encode_cls(dev_triples)
test_data  = encode_cls(test_triples)


batch_size = 512
dev_loader = DataLoader(TensorDataset(dev_data), batch_size=batch_size)
train_loader = DataLoader(TensorDataset(train_data), batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(TensorDataset(test_data),  batch_size=batch_size)


model = RGCNClassifier(num_nodes, num_rels, hidden_dim=100, num_layers=2).to(device)
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)



In [ ]:

for epoch in range(1, 201):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch[0].to(device)
        h_idx, t_idx, r_idx = batch[:,0], batch[:,1], batch[:,2]
        # input features: identity matrix for one-hot init
        x = torch.eye(num_nodes, device=device)
        logits = model(x, data.edge_index, data.edge_type, h_idx, t_idx)
        loss = F.cross_entropy(logits, r_idx)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        total_loss += loss.item() * h_idx.size(0)
    if epoch % 10 == 0:
        avg_loss = total_loss / len(train_data)
        print(f"Epoch {epoch:02d} | Train Loss: {avg_loss:.4f}")




Epoch 10 | Train Loss: 0.5060
Epoch 20 | Train Loss: 0.2101
Epoch 30 | Train Loss: 0.1853
Epoch 40 | Train Loss: 0.1473
Epoch 50 | Train Loss: 0.1213
Epoch 60 | Train Loss: 0.0777
Epoch 70 | Train Loss: 0.0504
Epoch 80 | Train Loss: 0.0369
Epoch 90 | Train Loss: 0.0259
Epoch 100 | Train Loss: 0.0220
Epoch 110 | Train Loss: 0.0182
Epoch 120 | Train Loss: 0.0152
Epoch 130 | Train Loss: 0.0121
Epoch 140 | Train Loss: 0.0118
Epoch 150 | Train Loss: 0.0124
Epoch 160 | Train Loss: 0.0066
Epoch 170 | Train Loss: 0.0084
Epoch 180 | Train Loss: 0.0071
Epoch 190 | Train Loss: 0.0090
Epoch 200 | Train Loss: 0.0073


In [ ]:

def eval_relation_metrics(data_tensor, ks=(1,3,10)):
    model.eval()
    with torch.no_grad():
        x = torch.eye(num_nodes, device=device)
        node_emb = x
        for conv in model.convs:
            node_emb = conv(node_emb, data.edge_index, data.edge_type)
            node_emb = F.relu(node_emb)
    correct = 0
    ranks = []
    with torch.no_grad():
        for h, t, r in data_tensor:
            h_idx = torch.tensor([h], device=device)
            t_idx = torch.tensor([t], device=device)
            combined = torch.cat([node_emb[h_idx], node_emb[t_idx]], dim=-1)
            scores = model.classifier(combined).squeeze(0)
            pred = scores.argmax().item()
            correct += (pred == r)
            _, idxs = scores.sort(descending=True)
            rank = (idxs == r).nonzero(as_tuple=False).item() + 1
            ranks.append(rank)
    acc = correct / len(data_tensor)
    ranks = torch.tensor(ranks, dtype=torch.float)
    mrr = (1.0/ranks).mean().item()
    hits = {f"Hits@{k}": (ranks<=k).float().mean().item() for k in ks}
    return acc, mrr, hits

dev_acc, dev_mrr, dev_hits = eval_relation_metrics(dev_data)
test_acc, test_mrr, test_hits = eval_relation_metrics(test_data)
print(f"Dev ▶ Acc={dev_acc:.4f} MRR={dev_mrr:.4f} Hits@1={dev_hits['Hits@1']:.4f} Hits@3={dev_hits['Hits@3']:.4f}")
print(f"Test ▶ Acc={test_acc:.4f} MRR={test_mrr:.4f} Hits@1={test_hits['Hits@1']:.4f} Hits@3={test_hits['Hits@3']:.4f}")


Dev ▶ Acc=0.6739 MRR=0.8188 Hits@1=0.6739 Hits@3=0.9565
Test ▶ Acc=0.6739 MRR=0.7919 Hits@1=0.6739 Hits@3=0.9130
